In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv1D, MaxPooling1D, BatchNormalization,
    Flatten, Dense, Dropout, concatenate
)
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

TRAIN_CSV = "/content/trainingData.csv"
VAL_CSV   = "/content/validationData.csv"

def load_and_prep(path, scaler=None, encoder=None, fit=False):
    df = pd.read_csv(path)
    df.replace(100, -110, inplace=True)
    X = df.iloc[:, :520].values
    y_env = df[['BUILDINGID', 'FLOOR']].astype(str).agg('_'.join, axis=1)
    y_loc = df[['LONGITUDE','LATITUDE']].values.astype(np.float32)

    if fit:
        scaler = StandardScaler().fit(X)
        encoder = LabelEncoder().fit(y_env)
    X = scaler.transform(X)
    X = X.reshape(-1, 520, 1)
    y_env_enc = encoder.transform(y_env)
    y_env_cat = to_categorical(y_env_enc)

    return X, y_env_cat, y_loc, scaler, encoder

# load train
X_train, y_env_train, y_loc_train, scaler, encoder = load_and_prep(TRAIN_CSV, fit=True)
# load validation
X_val, y_env_val, y_loc_val, _, _ = load_and_prep(VAL_CSV, scaler, encoder, fit=False)

# === 4. build multi‐task model ===
inp = Input(shape=(520,1))
x = Conv1D(64, 5, activation='relu', padding='same')(inp)
x = BatchNormalization()(x)
x = MaxPooling1D(2)(x)
x = Conv1D(128, 3, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling1D(2)(x)
x = Conv1D(256, 3, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling1D(2)(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)

env_out = Dense(y_env_train.shape[1], activation='softmax', name='env_id')(x)
loc_out = Dense(128, activation='relu')(x)
loc_out = Dropout(0.2)(loc_out)
loc_out = Dense(2, activation='linear', name='loc')(loc_out)

model = Model(inputs=inp, outputs=[env_out, loc_out])
model.summary()

model.compile(
    optimizer='adam',
    loss={
      'env_id':'categorical_crossentropy',
      'loc':'mse'
    },
    loss_weights={'env_id':1.0, 'loc':0.5},
    metrics={'env_id':'accuracy', 'loc':'mae'}
)

es = EarlyStopping(
    monitor='val_env_id_accuracy',
    mode='max',
    patience=5,
    restore_best_weights=True
)
rlr = ReduceLROnPlateau(
    monitor='val_env_id_accuracy',
    mode='max',
    factor=0.5,
    patience=3
)

history = model.fit(
    X_train,
    {'env_id':y_env_train, 'loc':y_loc_train},
    validation_data=(X_val, {'env_id':y_env_val, 'loc':y_loc_val}),
    epochs=50,
    batch_size=32,
    callbacks=[es, rlr]
)

evals = model.evaluate(
    X_val, {'env_id':y_env_val, 'loc':y_loc_val},
    batch_size=32, return_dict=True
)
print(f"Val Env Acc: {evals['env_id_accuracy']*100:.2f}%")
print(f"Val Loc  MAE: {evals['loc_mae']:.2f} meters")

env_pred_proba, loc_pred = model.predict(X_val)
env_pred = encoder.inverse_transform(env_pred_proba.argmax(axis=1))

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 520, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 520, 64)   │        384 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 520, 64)   │        256 │ conv1d_3[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_3     │ (None, 260, 64)   │          0 │ batch_normalizat… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 260, 128)  │     24,704 │ max_pooling1d_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 260, 128)  │        512 │ conv1d_4[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_4     │ (None, 130, 128)  │          0 │ batch_normalizat… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_5 (Conv1D)   │ (None, 130, 256)  │     98,560 │ max_pooling1d_4[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 130, 256)  │      1,024 │ conv1d_5[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_5     │ (None, 65, 256)   │          0 │ batch_normalizat… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 16640)     │          0 │ max_pooling1d_5[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 256)       │  4,260,096 │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256)       │      1,024 │ dense_2[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 256)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 128)       │     32,896 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 128)       │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ env_id (Dense)      │ (None, 13)        │      3,341 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ loc (Dense)         │ (None, 2)         │        258 │ dropout_3[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,423,055 (16.87 MB)

 Trainable params: 4,421,647 (16.87 MB)

 Non-trainable params: 1,408 (5.50 KB)

Epoch 1/50
624/624 ━━━━━━━━━━━━━━━━━━━━ 156s 240ms/step - env_id_accuracy: 0.1667 - env_id_loss: 2.6130 - loc_loss: 11821350125568.0000 - loc_mae: 2434236.7500 - loss: 5910675062784.0000 - val_env_id_accuracy: 0.1863 - val_env_id_loss: 2.2828 - val_loc_loss: 11636003831808.0000 - val_loc_mae: 2412430.2500 - val_loss: 5817605554176.0000 - learning_rate: 0.0010
Epoch 2/50
624/624 ━━━━━━━━━━━━━━━━━━━━ 199s 236ms/step - env_id_accuracy: 0.1759 - env_id_loss: 2.6600 - loc_loss: 11485386375168.0000 - loc_mae: 2396979.5000 - loss: 5742693187584.0000 - val_env_id_accuracy: 0.0333 - val_env_id_loss: 2.5540 - val_loc_loss: 11310670544896.0000 - val_loc_mae: 2380363.0000 - val_loss: 5655722196992.0000 - learning_rate: 0.0010
Epoch 3/50
624/624 ━━━━━━━━━━━━━━━━━━━━ 148s 237ms/step - env_id_accuracy: 0.1618 - env_id_loss: 2.5220 - loc_loss: 10121540272128.0000 - loc_mae: 2248413.7500 - loss: 5060770136064.0000 - val_env_id_accuracy: 0.1125 - val_env_id_loss: 2.3070 - val_loc_loss: 9169901977600.000

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, MaxPooling1D
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import seaborn as sns

train_file_path = "/content/trainingData.csv"
df_train = pd.read_csv(train_file_path)
df_train.replace(100, -110, inplace=True)

X_train = df_train.iloc[:, :520].values
y_train_env = df_train[['BUILDINGID', 'FLOOR']].astype(str).agg('_'.join, axis=1)
y_train_loc = df_train[['LONGITUDE', 'LATITUDE']].values

encoder = LabelEncoder()
y_train_env_encoded = encoder.fit_transform(y_train_env)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

X_train_env = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

num_classes = len(np.unique(y_train_env_encoded))
y_train_env_cat = to_categorical(y_train_env_encoded, num_classes=num_classes)

model_env = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(520, 1)),
    MaxPooling1D(pool_size=2),
    Conv1D(128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])
model_env.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_env.fit(X_train_env, y_train_env_cat, epochs=20, batch_size=32, validation_split=0.1)

unique_envs = np.unique(y_train_env_encoded)
localization_models = {}

for env in unique_envs:
    mask = (y_train_env_encoded == env)
    X_env = X_train[mask]
    y_env = y_train_loc[mask]
    knn = KNeighborsRegressor(n_neighbors=5, metric='euclidean')
    knn.fit(X_env, y_env)
    localization_models[env] = knn

val_file_path = "/content/validationData.csv"
df_val = pd.read_csv(val_file_path)
df_val.replace(100, -110, inplace=True)

X_val = df_val.iloc[:, :520].values
y_val_env = df_val[['BUILDINGID', 'FLOOR']].astype(str).agg('_'.join, axis=1)
y_val_loc = df_val[['LONGITUDE', 'LATITUDE']].values

y_val_env_encoded = encoder.transform(y_val_env)
y_val_env_cat = to_categorical(y_val_env_encoded, num_classes=num_classes)

X_val = scaler.transform(X_val)
X_val_env = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)

env_loss, env_acc = model_env.evaluate(X_val_env, y_val_env_cat)
print(f"Validation Environment Identification Accuracy: {env_acc*100:.2f}%")

y_pred_env_val = model_env.predict(X_val_env)
y_pred_env_classes = np.argmax(y_pred_env_val, axis=1)

mae_list = []
for i in range(len(X_val)):
    env_pred = y_pred_env_classes[i]
    if env_pred in localization_models:
        loc_model = localization_models[env_pred]
        y_pred = loc_model.predict(X_val[i].reshape(1, -1))
        mae = np.mean(np.abs(y_pred - y_val_loc[i]))
        mae_list.append(mae)

overall_mae = np.mean(mae_list)
print(f"Validation Indoor Localization Mean Absolute Error: {overall_mae:.2f} meters")

plt.figure(figsize=(12, 6))
sns.heatmap(df_train.iloc[:, :520].replace(-110, np.nan), cmap="coolwarm", cbar=True)
plt.xlabel("WiFi Access Points (WAPs)")
plt.ylabel("Samples")
plt.title("WiFi RSSI Signal Strength Heatmap")
plt.savefig('wifi_rssi_heatmap.png')
plt.close()

plt.figure(figsize=(10, 6))
plt.scatter(df_train['LONGITUDE'], df_train['LATITUDE'], alpha=0.5, c=df_train['FLOOR'], cmap='viridis')
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("Indoor Position Distribution (Color: Floor Level)")
plt.colorbar(label="Floor Level")
plt.savefig('indoor_position_distribution.png')
plt.close()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
561/561 ━━━━━━━━━━━━━━━━━━━━ 63s 109ms/step - accuracy: 0.8789 - loss: 0.4423 - val_accuracy: 0.8636 - val_loss: 0.5968
Epoch 2/20
561/561 ━━━━━━━━━━━━━━━━━━━━ 46s 82ms/step - accuracy: 0.9823 - loss: 0.0531 - val_accuracy: 0.8816 - val_loss: 0.7419
Epoch 3/20
561/561 ━━━━━━━━━━━━━━━━━━━━ 48s 86ms/step - accuracy: 0.9884 - loss: 0.0330 - val_accuracy: 0.8907 - val_loss: 0.9341
Epoch 4/20
561/561 ━━━━━━━━━━━━━━━━━━━━ 45s 80ms/step - accuracy: 0.9929 - loss: 0.0208 - val_accuracy: 0.8686 - val_loss: 0.9492
Epoch 5/20
561/561 ━━━━━━━━━━━━━━━━━━━━ 83s 82ms/step - accuracy: 0.9932 - loss: 0.0212 - val_accuracy: 0.8852 - val_loss: 0.9096
Epoch 6/20
561/561 ━━━━━━━━━━━━━━━━━━━━ 83s 84ms/step - accuracy: 0.9942 - loss: 0.0185 - val_accuracy: 0.8751 - val_loss: 1.4166
Epoch 7/20
561/561 ━━━━━━━━━━━━━━━━━━━━ 80s 80ms/step - accuracy: 0.9943 - loss: 0.0168 - val_accuracy: 0.8771 - val_loss: 1.4351
Epoch 8/20
561/561 ━━━━━━━━━━━━━━━━━━━━ 83s 82ms/step - accuracy: 0.9939 - loss: 0.0156 -